# Automated Web Scrape Practice

In [1]:
# Imports
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

In [2]:
# Set up Splinter
browser = Browser('chrome')

In [3]:
# Visit the Quotes to Scrape site
url = 'https://kansascity.craigslist.org/search/syp#search=1~gallery~0~0'
browser.visit(url)

In [4]:
# Parse the HTML
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [24]:
# Scrape the heading
list_names = soup.find_all('a', class_='cl-app-anchor text-only posting-title')
card_infos = soup.find_all('div', class_='gallery-card')

meta_soup = BeautifulSoup(str(card_infos), 'html.parser')
meta_infos = meta_soup.find_all('div', class_='meta')
price_infos = meta_soup.find_all('span', class_='priceinfo')

posting_time = []
posting_title = []
posting_price = []
posting_city = []
posting_state = []
posting_combined_location = []
search_type = []


for minfo in meta_infos:
    # Get Posting Time
    posting_time.append(minfo.text.split('·', 1)[0])
    
    # Get Posting Location
    posting_location = minfo.text.split('·', 1)[1]

    # Posting City and State Combined
    posting_combined_location.append(posting_location)

    # Split Posting City and State
    posting_city.append(posting_location.split(',', 1)[0])
    if len(posting_location.split(',', 1)) > 1:
        posting_state.append(posting_location.split(',', 1)[1])
    else:
        posting_state.append('')
    

for list_name in list_names:
    search_type.append('Computer Parts')
    posting_title.append(list_name.text)


for price_info in card_infos:
    # Strip '$' in case we want to use the value only
    try: 
        splt_price = price_info.find('span', class_='priceinfo').text
    except:
        splt_price = '$0'
        
    posting_price.append(splt_price)


# Create the DataFrame
list_info_df = pd.DataFrame({'List_Type':search_type, 'Price': posting_price, 'Item_Title': posting_title, 'Time_Listed': posting_time, 'Location':posting_combined_location})
list_info_df.tail(10)

,List_Type,Price,Item_Title,Time_Listed,Location
100,Computer Parts,$5,"Sony DS, DD 5.25 diskettes",5/5,Kansas City MO North
101,Computer Parts,$75,"47"" 3d tv vizio or hard drives.",5/4,Shawnee
102,Computer Parts,$1,"FujiFilm DS, HD 5.25 diskette",5/4,Kansas City MO North
103,Computer Parts,$10,"3M Highland DS, DD 5.25"" diskettes",5/4,Kansas City MO North
104,Computer Parts,$20,Netgear EX7000 Wi-Fi Extender,5/3,Peculiar
105,Computer Parts,$25,"2 TB, 1 TB, 500 GB hard drives",5/1,Kansas City
106,Computer Parts,$35,MSI 2GB video card,5/1,Kansas City
107,Computer Parts,$1,AMD 3600 - ASROCK B550M PRO4 - LIAN LI - Corsa...,4/27,Grandview
108,Computer Parts,$99,Netgear GS110TP-300NAS POE+ Managed Network Sw...,4/27,Grandview
109,Computer Parts,$90,Thermaltake TH420 V2 Ultra. AIO liquid Cooler,4/26,Odessa


In [25]:
# Create CSV file to Output folder
list_info_df.to_csv('Output/kansascity_craigslist_computer_parts.csv',index=False)

In [8]:
# End the automated browsing section
browser.quit()